## Time series
Plik w którym tworze jeden odpowiedni szereg czasowy

In [2]:
import pandas as pd
import numpy as np

In [5]:
names = ["AMZN", "AAPL", "MSFT", "TSLA", "GOOG", "GOOGL"]
#names = ["TSLA"]
#company = "AMZN"
#weight = "w1"
for company in names:
    # read in data
    df = pd.read_csv(f"../datasets/v3/senti/{company}.csv", parse_dates=["post_date"])
    # remove
    df = df.set_index('post_date')
    #df = df.drop(index=df.between_time('00:00', '15:00').index)
    stocks = pd.read_csv(f'../datasets/stocks_clean/{company}.csv', parse_dates=['date_'])
    stocks.rename(columns={"date_": "post_date"}, inplace=True)
    stocks = stocks.set_index("post_date")
    test = stocks.merge(df, how='outer', left_index=True, right_index=True)
    test = test.dropna().reset_index()
    test.to_csv(f'../datasets/v3/stocked/{company}.csv', index=False)

In [ ]:
df = pd.read_csv(
    f"../datasets/v3/more_cols_with_stock/w1/{company}.csv", parse_dates=["post_date"]
)

In [600]:
df = df.set_index('post_date')
closed_market = df.between_time('00:00', '15:00')

In [601]:
closed_market_grp = closed_market.groupby(pd.Grouper(freq='D'))
closed_market = closed_market_grp.mean()
closed_market["count"] = closed_market_grp['count'].sum()
closed_market.head()

,comment_num_sum,comp_max,comp_mean,comp_median,comp_min,comp_std,like_num_sum,neg_max,neg_mean,neg_median,...,neu_min,neu_std,count,pos_max,pos_mean,pos_median,pos_min,pos_std,retweet_num_sum,relative_count
post_date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,3.153846,8.020475,-6.783037,-0.251634,-112.403060,34.769896,10.923077,55.090596,4.320026,0.152269,...,0.666762,42.859360,86,4.226804,1.076703,0.049533,0.000000,2.095665,53.000000,0.263182
2015-01-02,0.312500,4.759035,0.505341,0.025283,-0.371374,1.771936,2.312500,0.213409,0.087958,0.059781,...,0.718069,6.577740,94,2.529366,0.351097,0.071967,0.030134,0.844288,22.062500,0.139304
2015-01-03,0.230769,0.372112,0.141817,0.184204,-0.189415,0.253211,2.230769,0.182435,0.067001,0.039808,...,0.774700,0.316820,41,0.208031,0.104499,0.099144,0.029077,0.090733,0.692308,0.081064
2015-01-04,0.071429,0.322478,0.062985,0.084136,-0.217061,0.285608,2.214286,0.207261,0.073351,0.023625,...,0.811932,0.813210,45,0.259632,0.114061,0.065621,0.030296,0.127796,1.571429,0.096398
2015-01-05,0.600000,1.005709,0.302556,0.268753,-0.217776,0.356436,4.000000,0.142903,0.019251,0.000000,...,0.675787,0.428728,152,0.484373,0.151867,0.126533,0.033933,0.139203,1.866667,0.147293


In [602]:
closed_market.index +=  pd.to_timedelta(1, unit='H')

In [603]:
len(closed_market)

1827

In [604]:
print(len(df))
df = df.drop(index=df.between_time('00:00', '15:00').index)
print(len(df))

41059
14287


In [605]:
#first_hour_mask = df.groupby(df.index.date).head(1).index

In [606]:
# print(len(first_hour_mask), len(closed_market))
# to_drop = list(set(closed_market.index.strftime("%Y-%m-%d")) - set(first_hour_mask.strftime("%Y-%m-%d")))
# print(to_drop)
# for i in to_drop:
#     closed_market.drop(closed_market.loc[i].index, inplace=True)
# closed_market.index = first_hour_mask
# print(len(first_hour_mask), len(closed_market))

1821 1827
['2020-01-01', '2019-06-19', '2019-04-25', '2017-10-26', '2017-05-30', '2017-10-18']
1821 1821


In [607]:
closed_market.head()

,comment_num_sum,comp_max,comp_mean,comp_median,comp_min,comp_std,like_num_sum,neg_max,neg_mean,neg_median,...,neu_min,neu_std,count,pos_max,pos_mean,pos_median,pos_min,pos_std,retweet_num_sum,relative_count
post_date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 16:00:00,3.153846,8.020475,-6.783037,-0.251634,-112.403060,34.769896,10.923077,55.090596,4.320026,0.152269,...,0.666762,42.859360,86,4.226804,1.076703,0.049533,0.000000,2.095665,53.000000,0.263182
2015-01-02 16:00:00,0.312500,4.759035,0.505341,0.025283,-0.371374,1.771936,2.312500,0.213409,0.087958,0.059781,...,0.718069,6.577740,94,2.529366,0.351097,0.071967,0.030134,0.844288,22.062500,0.139304
2015-01-03 16:00:00,0.230769,0.372112,0.141817,0.184204,-0.189415,0.253211,2.230769,0.182435,0.067001,0.039808,...,0.774700,0.316820,41,0.208031,0.104499,0.099144,0.029077,0.090733,0.692308,0.081064
2015-01-04 16:00:00,0.071429,0.322478,0.062985,0.084136,-0.217061,0.285608,2.214286,0.207261,0.073351,0.023625,...,0.811932,0.813210,45,0.259632,0.114061,0.065621,0.030296,0.127796,1.571429,0.096398
2015-01-05 16:00:00,0.600000,1.005709,0.302556,0.268753,-0.217776,0.356436,4.000000,0.142903,0.019251,0.000000,...,0.675787,0.428728,152,0.484373,0.151867,0.126533,0.033933,0.139203,1.866667,0.147293


In [608]:
# for col in df.columns:
#     df.loc[first_hour_mask, col] = (df.loc[first_hour_mask, col] + closed_market[col])/2

In [609]:
df.head()

,comment_num_sum,comp_max,comp_mean,comp_median,comp_min,comp_std,like_num_sum,neg_max,neg_mean,neg_median,...,neu_min,neu_std,count,pos_max,pos_mean,pos_median,pos_min,pos_std,retweet_num_sum,relative_count
post_date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 16:00:00,1.576923,4.111387,-3.428879,-0.125817,-56.564200,17.533267,7.961538,27.731148,2.186563,0.076135,...,0.623181,21.516805,46.5,2.163902,0.552616,0.024766,0.0000,1.072196,27.0,0.197629
2015-01-01 17:00:00,0.000000,4.734320,1.255111,0.367658,-0.449190,2.370304,14.000000,0.130200,0.032550,0.000000,...,0.689850,4.841415,4.0,1.835200,0.548837,0.180075,0.0000,0.874219,8.0,0.100000
2015-01-01 18:00:00,1.000000,0.771855,0.092755,0.000000,-0.725340,0.518459,14.000000,0.351750,0.068619,0.000000,...,0.604800,0.137948,13.0,0.249700,0.096435,0.093450,0.0000,0.101562,0.0,0.250000
2015-01-01 19:00:00,1.000000,0.676200,0.304509,0.438900,-0.458160,0.361174,4.000000,0.229200,0.046843,0.000000,...,0.774900,0.041697,7.0,0.275100,0.184971,0.163800,0.1188,0.067042,0.0,0.162791
2015-01-01 20:00:00,0.000000,0.438900,-0.005399,0.000000,-0.553035,0.424411,5.000000,0.364350,0.116550,0.000000,...,0.533400,0.196248,5.0,0.151200,0.082720,0.126000,0.0000,0.076042,0.0,0.151515


In [610]:

stocks = pd.read_csv(f'../datasets/stocks_clean/{company}.csv', parse_dates=['date_'])
stocks.rename(columns={"date_": "post_date"}, inplace=True)
stocks = stocks.set_index("post_date")

In [611]:
stocks.head()

,ticker,open,high,low,close,vol
post_date,,,,,,
2015-01-02 17:00:00,AMZN,312.58,314.75,312.1100,313.52,404201
2015-01-02 18:00:00,AMZN,313.25,313.59,308.6300,308.96,509208
2015-01-02 19:00:00,AMZN,308.84,310.12,306.9601,309.75,427854
2015-01-02 20:00:00,AMZN,309.77,309.93,308.0500,308.40,209933
2015-01-02 21:00:00,AMZN,308.41,308.48,307.2500,308.35,158209


In [612]:
#stocks["diffs"] = stocks.close.pct_change() * 100
#stocks["bin_2"] = pd.cut(stocks.diffs, bins=[-100, 0, 100], labels=[-1, 1])
#stocks["bin_3"] = pd.cut(stocks.diffs, bins=[-100, -0.25, 0.25, 100], labels=[-1, 0, 1])
#stocks["bin_5"] = pd.cut(stocks.diffs, bins=[-100, -0.75, -0.25, 0.25, 0.75, 100], labels=[-2, -1, 0, 1, 2])

In [613]:
test = stocks.merge(df, how='outer', left_index=True, right_index=True)

In [614]:
test = test.dropna().reset_index()

In [615]:
test.to_csv(f'../datasets/v3/more_cols_with_stock/w1/{company}.csv', index=False)